<a href="https://colab.research.google.com/github/yilmajung/belief_and_llms_v0/blob/main/1_extract_persona_vectors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase 1: Extract Persona Vectors (Layers 5-20)

This notebook extracts demographic steering vectors from **multiple layers (5-20)** of Meta-Llama-3-8B-Instruct.

**Output:** One vector file per layer:
- `gss_demographic_vectors_layer5.pt`
- `gss_demographic_vectors_layer6.pt`
- ...
- `gss_demographic_vectors_layer20.pt`

In [ ]:
!pip install -q -U bitsandbytes

In [ ]:
import torch
import torch.nn.functional as F
import numpy as np
import os

In [ ]:
# Link to Google Drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Load extraction_datasets.json
import json
with open("/content/drive/MyDrive/belief_and_llms_v0/gss_extraction_datasets.json", "r") as f:
    extraction_datasets = json.load(f)

print(f"Loaded {len(extraction_datasets)} demographic groups.")
print("Groups:", list(extraction_datasets.keys())[:5], "...")

In [ ]:
# Model Setup
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

# Load model in 4-bit to save memory
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Configuration: Layers to extract
LAYERS_TO_EXTRACT = list(range(5, 21))  # Layers 5, 6, 7, ..., 20
SAVE_DIR = "/content/drive/MyDrive/belief_and_llms_v0/vectors"
os.makedirs(SAVE_DIR, exist_ok=True)

print(f"Will extract vectors from layers: {LAYERS_TO_EXTRACT}")
print(f"Save directory: {SAVE_DIR}")

In [ ]:
# Helper: Hook Function

def get_layer_activations(model, tokenizer, inputs_text, layer_idx):
    """
    Extract hidden state activations at a specific layer.
    Returns the activation at the last token position.
    """
    inputs = tokenizer(inputs_text, return_tensors="pt").to(model.device)
    captured_hidden = None

    def hook_fn(module, input, output):
        nonlocal captured_hidden

        # Unwrap tuple if needed
        if isinstance(output, tuple):
            h_states = output[0]
        else:
            h_states = output

        # Handle different tensor shapes
        if h_states.dim() == 3:
            # [Batch, Seq, Hidden] -> take last token
            captured_hidden = h_states[0, -1, :].detach().cpu()
        elif h_states.dim() == 2:
            # [Seq, Hidden] -> take last token
            captured_hidden = h_states[-1, :].detach().cpu()
        else:
            raise ValueError(f"Unexpected tensor shape: {h_states.shape}")

    # Register hook and run forward pass
    layer = model.model.layers[layer_idx]
    handle = layer.register_forward_hook(hook_fn)

    with torch.no_grad():
        model(**inputs)

    handle.remove()
    return captured_hidden

In [ ]:
def extract_vectors_for_layer(model, tokenizer, extraction_datasets, layer_idx):
    """
    Extract all demographic vectors for a single layer.
    
    Returns:
        dict: {demographic_label: {'vector': tensor, 'magnitude': float}}
    """
    demographic_vectors = {}
    
    for label, pairs in extraction_datasets.items():
        # Unzip pairs into positive and negative texts
        pos_texts = [p[0] for p in pairs]
        neg_texts = [p[1] for p in pairs]

        # Get activations for X+ (persona)
        pos_acts = []
        for text in pos_texts:
            act = get_layer_activations(model, tokenizer, text, layer_idx)
            pos_acts.append(act)

        # Get activations for X- (baseline)
        neg_acts = []
        for text in neg_texts:
            act = get_layer_activations(model, tokenizer, text, layer_idx)
            neg_acts.append(act)

        # Stack and compute mean
        pos_tensor = torch.vstack(pos_acts)
        neg_tensor = torch.vstack(neg_acts)

        # Calculate difference vector: mean(X+) - mean(X-)
        diff_vector = torch.mean(pos_tensor, dim=0) - torch.mean(neg_tensor, dim=0)

        # Normalize
        raw_magnitude = torch.norm(diff_vector).item()
        if raw_magnitude > 0:
            normalized_vector = diff_vector / torch.norm(diff_vector)
        else:
            normalized_vector = diff_vector  # Keep zero vector as is

        # Store result
        demographic_vectors[label] = {
            "vector": normalized_vector,
            "magnitude": raw_magnitude
        }

    return demographic_vectors

In [ ]:
# Main Extraction Loop: Extract vectors for ALL layers

all_layer_vectors = {}  # Store all results in memory

for layer_idx in LAYERS_TO_EXTRACT:
    print(f"\n{'='*60}")
    print(f"Extracting vectors from Layer {layer_idx}...")
    print(f"{'='*60}")
    
    # Extract vectors for this layer
    vectors = extract_vectors_for_layer(model, tokenizer, extraction_datasets, layer_idx)
    
    # Store in memory
    all_layer_vectors[layer_idx] = vectors
    
    # Save to file
    save_path = os.path.join(SAVE_DIR, f"gss_demographic_vectors_layer{layer_idx}.pt")
    torch.save(vectors, save_path)
    
    # Print summary
    magnitudes = [v['magnitude'] for v in vectors.values()]
    print(f"  Saved {len(vectors)} vectors to {save_path}")
    print(f"  Magnitude range: {min(magnitudes):.4f} - {max(magnitudes):.4f}")

print(f"\n{'='*60}")
print("EXTRACTION COMPLETE!")
print(f"{'='*60}")
print(f"Extracted vectors from {len(LAYERS_TO_EXTRACT)} layers.")
print(f"Files saved to: {SAVE_DIR}")

## Verification: Sanity Checks Across Layers

In [ ]:
# Verify: Check Democrat vs Republican similarity across layers

def check_similarity(vectors_dict, label_a, label_b):
    """Compute cosine similarity between two demographic vectors."""
    if label_a not in vectors_dict or label_b not in vectors_dict:
        return None
    vec_a = vectors_dict[label_a]["vector"]
    vec_b = vectors_dict[label_b]["vector"]
    return F.cosine_similarity(vec_a.unsqueeze(0), vec_b.unsqueeze(0)).item()

print("\n--- SANITY CHECK: Democrat vs Republican Across Layers ---")
print(f"{'Layer':<8} | {'Dem vs Rep':<12} | {'Black vs Rep':<12}")
print("-" * 40)

for layer_idx in LAYERS_TO_EXTRACT:
    vectors = all_layer_vectors[layer_idx]
    sim_pol = check_similarity(vectors, "PartyID_Strong Democrat", "PartyID_Strong Republican")
    sim_race = check_similarity(vectors, "Race_Black person", "PartyID_Strong Republican")
    print(f"{layer_idx:<8} | {sim_pol:+.4f}      | {sim_race:+.4f}")

## Visualization: Compare Vector Magnitudes Across Layers

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Build DataFrame of magnitudes
magnitude_data = []
for layer_idx in LAYERS_TO_EXTRACT:
    vectors = all_layer_vectors[layer_idx]
    for label, data in vectors.items():
        magnitude_data.append({
            'layer': layer_idx,
            'demographic': label,
            'category': label.split('_')[0],
            'magnitude': data['magnitude']
        })

df_mag = pd.DataFrame(magnitude_data)

# Plot: Average magnitude by layer
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
avg_by_layer = df_mag.groupby('layer')['magnitude'].mean()
plt.plot(avg_by_layer.index, avg_by_layer.values, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Layer', fontsize=12)
plt.ylabel('Average Magnitude', fontsize=12)
plt.title('Average Vector Magnitude by Layer', fontsize=14)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
# Heatmap: magnitude by demographic and layer
pivot = df_mag.pivot(index='demographic', columns='layer', values='magnitude')
# Show only a subset for readability
sample_demos = ['PartyID_Strong Republican', 'PartyID_Strong Democrat', 
                'Race_Black person', 'Race_White person',
                'PolViews_person with a conservative political view',
                'PolViews_person with a liberal political view']
pivot_subset = pivot.loc[[d for d in sample_demos if d in pivot.index]]
sns.heatmap(pivot_subset, cmap='YlOrRd', annot=True, fmt='.2f')
plt.title('Magnitude by Demographic & Layer', fontsize=14)
plt.xlabel('Layer')
plt.ylabel('')

plt.tight_layout()
plt.show()

## PCA Visualization for Each Layer

In [ ]:
from sklearn.decomposition import PCA

# Create PCA plots for layers 5, 12, and 20
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for idx, layer_idx in enumerate([5, 12, 20]):
    vectors = all_layer_vectors[layer_idx]
    
    # Prepare data
    labels = []
    vectors_list = []
    categories = []
    
    for key, data in vectors.items():
        vec = data['vector'].cpu().numpy()
        if not (np.isnan(vec).any() or np.isinf(vec).any()):
            vectors_list.append(vec)
            labels.append(key)
            categories.append(key.split('_')[0])
    
    X = np.array(vectors_list)
    
    # PCA
    pca = PCA(n_components=2)
    X_2d = pca.fit_transform(X)
    
    # Plot
    ax = axes[idx]
    df_plot = pd.DataFrame({
        'x': X_2d[:, 0],
        'y': X_2d[:, 1],
        'category': categories
    })
    
    for cat in df_plot['category'].unique():
        mask = df_plot['category'] == cat
        ax.scatter(df_plot.loc[mask, 'x'], df_plot.loc[mask, 'y'], 
                   label=cat, alpha=0.7, s=100)
    
    ax.axhline(0, color='black', linestyle='--', alpha=0.3)
    ax.axvline(0, color='black', linestyle='--', alpha=0.3)
    ax.set_title(f'Layer {layer_idx}', fontsize=14)
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%})')
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%})')
    if idx == 2:
        ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')

plt.suptitle('PCA of Demographic Vectors Across Layers', fontsize=16)
plt.tight_layout()
plt.show()

## Summary

This notebook extracted demographic steering vectors from layers 5-20.

**Output files:**
```
vectors/
├── gss_demographic_vectors_layer5.pt
├── gss_demographic_vectors_layer6.pt
├── ...
└── gss_demographic_vectors_layer20.pt
```

**Next steps:**
- Use `2_simulate_steering_vectors.ipynb` to test steering effectiveness
- Use `3_investigate_correlations.ipynb` to analyze how steering affects other demographics

**Important:** For steering experiments, always use vectors extracted at the SAME layer as the injection layer.